In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0,
    max_retries=2,
)
response = llm.invoke('The first person to land on moon was ...')
print(response)


content="The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 44, 'total_tokens': 87, 'completion_time': 0.038385374, 'prompt_time': 0.073796837, 'queue_time': 4.949512277, 'total_time': 0.112182211}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--3e242e3d-be3b-4ad9-b6ff-9e68fd4760f3-0' usage_metadata={'input_tokens': 44, 'output_tokens': 43, 'total_tokens': 87}


In [20]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://mycareer.verizon.com/jobs/r-1071918/full-stack-developer")
page_data = loader.load().pop().page_content
print(page_data)















Full Stack Developer, Branchburg, New Jersey, +3 other locations | Verizon Careers








































Skip to main content







About Us


Careers



















Saved Jobs
0












Careers



                        Jobs
                    


Life at Verizon

Life at Verizon
Benefits
Our Culture
Culture of Learning
Doing Good
FAQs
How We Hire
Military
Students & Internships



Career Paths

Career Paths
Corporate
Customer Support
Cybersecurity
Field Operations & Technicians
Marketing
Retail
Sales
Technology



Locations

Locations
APAC
EMEA
India
North America




                        Events
                    


Blog

Blog
Our Culture
Verizon Voices
Ask a Recruiter




                        Candidate Home
                    


Join our talent network






Saved Jobs
0



About Us














Home



Jobs



Full Stack Developer












Cybersecurity
Full Stack Developer

Branchburg, New Jersey
Southlake, Texas
Ashburn, Virg

In [17]:
from langchain_core.prompts import PromptTemplate

In [ ]:
prompt_extract = PromptTemplate.from_template(
    '''
    "Extract the relevant information from the following text: {input_text}
    ### Instruction:
    The scraped text is from the career's page of a website.
    your task is to extract the relevant information from the text in json format containing following keys:
    'role', 'experience', 'skills', 'description'.
    only return the valid JSON.
    ### VALID JSON (NO PREAMBLE)
    '''
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'input_text': page_data})
res.content


'{\n  "role": "Principal Full Stack Developer",\n  "experience": {\n    "required": "Bachelor’s degree or four or more years of work experience",\n    "preferred": [\n      "Six more years of relevant work experience",\n      "Experience with Linux/RHEL Operating Systems",\n      "Experience with CI/CD pipelines and container-based development and orchestration",\n      "Experience with database administration & management:  (e.g. Oracle SQL, MySQL, MS SQL etc.)",\n      "Experience with code versioning tools like Git"\n    ]\n  },\n  "skills": {\n    "required": [\n      "TypeScript and Python",\n      "React, Nest.js, Next.js, HTML",\n      "Node.js, Python, and Java"\n    ],\n    "preferred": [\n      "Linux/RHEL Operating Systems",\n      "CI/CD pipelines and container-based development and orchestration",\n      "database administration & management:  (e.g. Oracle SQL, MySQL, MS SQL etc.)",\n      "code versioning tools like Git"\n    ]\n  },\n  "description": "Join Verizon in thi

In [28]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Principal Full Stack Developer',
 'experience': {'required': 'Bachelor’s degree or four or more years of work experience',
  'preferred': ['Six more years of relevant work experience',
   'Experience with Linux/RHEL Operating Systems',
   'Experience with CI/CD pipelines and container-based development and orchestration',
   'Experience with database administration & management:  (e.g. Oracle SQL, MySQL, MS SQL etc.)',
   'Experience with code versioning tools like Git']},
 'skills': {'required': ['TypeScript and Python',
   'React, Nest.js, Next.js, HTML',
   'Node.js, Python, and Java'],
  'preferred': ['Linux/RHEL Operating Systems',
   'CI/CD pipelines and container-based development and orchestration',
   'database administration & management:  (e.g. Oracle SQL, MySQL, MS SQL etc.)',
   'code versioning tools like Git']},
 'description': 'Join Verizon in this Principal Full Stack Developer position, which expands our Network Security Team with a role emphasis on full-sta

In [29]:
type(json_res)

dict

In [37]:
import pandas as pd
import uuid

In [32]:
data = pd.read_csv("../my_portfolio.csv")
data.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [38]:
import chromadb
client = chromadb.PersistentClient('vectorstore') 
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in data.iterrows():
        collection.add(
            documents=[row['Techstack']],
            metadatas=[{"links": row["Links"]}],
            ids=[str(uuid.uuid4())]
        )


In [42]:
links = collection.query(query_texts=["Experience in python", "Expert in machine learning", "experience in software development"],
 n_results=2).get("metadatas", [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [44]:
job = json_res
job['skills']

{'required': ['TypeScript and Python',
  'React, Nest.js, Next.js, HTML',
  'Node.js, Python, and Java'],
 'preferred': ['Linux/RHEL Operating Systems',
  'CI/CD pipelines and container-based development and orchestration',
  'database administration & management:  (e.g. Oracle SQL, MySQL, MS SQL etc.)',
  'code versioning tools like Git']}

In [45]:
job

{'role': 'Principal Full Stack Developer',
 'experience': {'required': 'Bachelor’s degree or four or more years of work experience',
  'preferred': ['Six more years of relevant work experience',
   'Experience with Linux/RHEL Operating Systems',
   'Experience with CI/CD pipelines and container-based development and orchestration',
   'Experience with database administration & management:  (e.g. Oracle SQL, MySQL, MS SQL etc.)',
   'Experience with code versioning tools like Git']},
 'skills': {'required': ['TypeScript and Python',
   'React, Nest.js, Next.js, HTML',
   'Node.js, Python, and Java'],
  'preferred': ['Linux/RHEL Operating Systems',
   'CI/CD pipelines and container-based development and orchestration',
   'database administration & management:  (e.g. Oracle SQL, MySQL, MS SQL etc.)',
   'code versioning tools like Git']},
 'description': 'Join Verizon in this Principal Full Stack Developer position, which expands our Network Security Team with a role emphasis on full-sta

In [ ]:
prompt_email = PromptTemplate.from_template(
    """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Faren, a business development executive at PPP. PPP is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of PPP
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase PPP's portfolio: {link_list}
        Remember you are Mohan, BDE at PPP. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
   )
email_prompt = prompt_email | llm
res = email_prompt.invoke(input={'job_description': str(job), 'link_list': links})
print(res.content)

Subject: Unlock Your Business Potential with PPP's Expertise

Dear Hiring Manager,

I came across the job description for the Principal Full Stack Developer position at Verizon, and I was impressed by the emphasis on full-stack software engineering & development. As a Business Development Executive at PPP, an AI & Software Consulting company, I believe our expertise can help you find the perfect candidate for this role.

At PPP, we specialize in facilitating the seamless integration of business processes through automated tools. Our team has empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. We have a proven track record of delivering high-quality projects on time and within budget.

Our portfolio showcases our expertise in various areas, including:

- Machine Learning and Python: Check out our recent projects at https://example.com/ml-python-portfolio and https://example.com/python-por

In [49]:
%pwd

'/Users/farnaznouri/Documents/Coding/genai-cold-email-generator/research'